# 第6章：流式输出和中断 - Streaming-steering

## 本章目标

本章将实现流式响应机制，让Agent能够实时输出执行过程，提升用户体验。你将学习：

- **三种stream模式**: values, updates, messages
- **事件级别流式输出**: astream_events
- **实时Token流式**: 逐字显示LLM输出
- **流式进度追踪**: 显示节点转换和工具调用

## 应用场景

- **实时对话**: 像ChatGPT一样逐字显示回答
- **长任务追踪**: 显示Agent执行进度
- **调试与监控**: 实时查看Agent内部状态


## 💡 这是 Claude Code 的什么功能？

这是 Claude Code 的**实时响应显示**和**执行控制**。你会看到：
- Token 一个个蹦出来（像打字机效果）
- 任务状态实时更新（"正在分析..." → "正在写入..."）
- 可以随时点击 **"Stop" 按钮中断执行**

**两个核心能力**：

**1. 流式输出**：
- 不用等 30 秒才看到结果，边执行边显示
- 用户知道 AI 当前在做什么（不是卡住了）

**2. 简单中断**：
- 用户点击停止按钮 → 立即停止当前执行
- 不保存状态，不恢复（和第 2 章的 interrupt 不同）
- 典型场景：发现 AI 方向错了，直接停止重新开始

**技术要点**：
- 使用 `astream_events` 实时获取执行事件
- 通过停止迭代来中断（而非 `interrupt()`）
- 三种流式模式：messages/updates/values

**与第 2 章 Human-in-the-Loop 的区别**：
- 第 2 章：AI 主动暂停，等待用户审批，可恢复
- 第 6 章：用户主动停止，直接终止，不恢复

本章实现的流式架构和中断控制，让 Claude Code 的用户体验更加流畅和可控。

## 1. 环境设置

In [ ]:
import os
import asyncio
import time
from typing import TypedDict, List, Annotated
from IPython.display import display, clear_output
import ipywidgets as widgets

# LangGraph
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import create_react_agent
from langgraph.checkpoint.memory import MemorySaver

# LangChain
from langchain_core.tools import tool
from langchain_core.messages import (
    HumanMessage, 
    SystemMessage, 
    AIMessage, 
    BaseMessage
)

# LLM
# from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatTongyi

In [ ]:
# 配置
os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_PROJECT"] = "chapter-06-streaming-output"

# 初始化LLM
# llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 选项2: 通义千问 (取消注释使用)
llm = ChatTongyi(model="qwen3-max", temperature=0)

print(f"✅ LLM初始化成功: {llm.model_name}")

## 2. 创建测试Agent

我们创建一个带有多个工具的Agent，用于演示流式输出。

### 为什么工具要有延迟？

下面定义的工具都包含 `time.sleep()` 延迟，这是为了：

1. **模拟真实环境**：实际的工具操作（数据库查询、API调用、文件操作等）都需要时间
2. **更好地演示流式输出**：如果工具瞬间完成，就看不到流式进度追踪的效果
3. **体验用户交互**：在长时间操作中，用户可以看到实时反馈，甚至可以中断操作

**在实际应用中**，你不需要添加这些人工延迟，真实的工具操作本身就有延迟。这些延迟只是为了让演示效果更加明显。

In [ ]:
# 定义模拟工具（有延迟，方便观察流式效果）
@tool
def search_database(query: str) -> str:
    """搜索数据库（模拟，有2秒延迟）
    
    Args:
        query: 搜索查询
    """
    time.sleep(2)  # 模拟数据库查询耗时
    return f"数据库搜索结果：找到关于'{query}'的3条记录\n1. 记录A\n2. 记录B\n3. 记录C"


@tool
def calculate(expression: str) -> str:
    """计算数学表达式（模拟，有1秒延迟）
    
    Args:
        expression: 数学表达式
    """
    time.sleep(1)  # 模拟计算耗时
    try:
        result = eval(expression)
        return f"计算结果: {expression} = {result}"
    except Exception as e:
        return f"计算错误: {e}"


@tool
def fetch_weather(city: str) -> str:
    """获取天气信息（模拟，有1.5秒延迟）
    
    Args:
        city: 城市名称
    """
    time.sleep(1.5)  # 模拟API调用耗时
    return f"{city}的天气：晴天，气温25°C，湿度60%"


tools = [search_database, calculate, fetch_weather]

print(f"✅ 定义了 {len(tools)} 个工具")
for t in tools:
    print(f"  - {t.name}")

In [ ]:
# 创建Agent（带checkpointer）
memory = MemorySaver()

agent = create_react_agent(
    llm,
    tools=tools,
    prompt="你是一个智能助手，可以搜索数据库、计算数学表达式和查询天气。请帮助用户解决问题。",
    checkpointer=memory
)

print("✅ Agent创建完成（带checkpointer）")

## 3. 模式一：stream_mode="values"

### 特点
- 返回每个节点执行后的**完整状态**
- 适合调试和完整状态追踪
- 数据量较大

### 使用场景
- 调试Agent行为
- 查看完整执行链路
- 记录完整状态日志

In [ ]:
print("="*60)
print("模式一：stream_mode='values'")
print("="*60)

config = {"configurable": {"thread_id": "test-values"}}
input_msg = {"messages": [HumanMessage(content="计算 15 * 23")]}

print("\n开始流式执行...\n")

step = 0
for chunk in agent.stream(input_msg, config, stream_mode="values"):
    step += 1
    print(f"\n--- Step {step} ---")
    
    # chunk是完整的state
    if "messages" in chunk:
        last_msg = chunk["messages"][-1]
        print(f"类型: {last_msg.__class__.__name__}")
        
        if hasattr(last_msg, 'content'):
            content = last_msg.content
            # 限制显示长度
            if len(content) > 100:
                print(f"内容: {content[:100]}...")
            else:
                print(f"内容: {content}")
        
        if hasattr(last_msg, 'tool_calls') and last_msg.tool_calls:
            print(f"工具调用: {last_msg.tool_calls[0]['name']}")

print("\n" + "="*60)
print("values模式演示完成")
print("="*60)

## 4. 模式二：stream_mode="updates"

### 特点
- 返回每个节点产生的**增量更新**
- 数据量较小，传输高效
- 需要自己维护完整状态

### 使用场景
- 网络传输优化
- 实时进度展示
- 增量更新UI

In [ ]:
print("="*60)
print("模式二：stream_mode='updates'")
print("="*60)

config = {"configurable": {"thread_id": "test-updates"}}
input_msg = {"messages": [HumanMessage(content="搜索'LangGraph教程'")]}

print("\n开始流式执行...\n")

for chunk in agent.stream(input_msg, config, stream_mode="updates"):
    # chunk是一个字典，key是节点名，value是该节点的更新
    for node_name, node_update in chunk.items():
        print(f"\n📍 节点: {node_name}")
        
        if "messages" in node_update:
            for msg in node_update["messages"]:
                print(f"  类型: {msg.__class__.__name__}")
                
                if hasattr(msg, 'content') and msg.content:
                    content = msg.content
                    if len(content) > 80:
                        print(f"  内容: {content[:80]}...")
                    else:
                        print(f"  内容: {content}")

print("\n" + "="*60)
print("updates模式演示完成")
print("="*60)

## 5. 模式三：stream_mode="messages"

### 特点
- 返回**Token级别**的流式输出
- 最接近真实对话体验（类似ChatGPT）
- 实时性最好

### 使用场景
- 聊天界面
- 实时打字效果
- 最佳用户体验

In [ ]:
print("="*60)
print("模式三：stream_mode='messages'")
print("="*60)

config = {"configurable": {"thread_id": "test-messages"}}
input_msg = {"messages": [HumanMessage(content="用一句话介绍LangGraph")]}

print("\n🤖 AI: ", end="", flush=True)

# messages模式返回的是消息流
for msg, metadata in agent.stream(input_msg, config, stream_mode="messages"):
    # 只显示AIMessage的内容
    if isinstance(msg, AIMessage) and msg.content:
        print(msg.content, end="", flush=True)

print("\n\n" + "="*60)
print("messages模式演示完成（实时Token流式）")
print("="*60)

## 6 模式四：astream_events

### 特点
- 返回**事件级别**的流式输出（比Token更细粒度）
- 可以监听LLM开始、Token流、工具调用等多种事件
- 最适合构建生产级应用
- 与LangChain事件标准兼容

### 使用场景
- 需要精确控制每个执行阶段
- 显示工具调用的开始和结束
- 构建类似Claude Code的交互体验
- 生产环境的监控和调试

In [ ]:
print("="*60)
print("模式四：astream_events（事件级流式）")
print("="*60)

config = {"configurable": {"thread_id": "test-events"}}
input_msg = {"messages": [HumanMessage(content="计算 20 * 15")]}

print("\n开始流式执行...\n")
all_events = []
# astream_events返回不同类型的事件
async for event in agent.astream_events(input_msg, config, version="v2"):
    all_events.append(event)
    event_type = event["event"]
    
    # 1. LLM开始生成
    if event_type == "on_chat_model_start":
        print("🤔 AI开始思考...")
    
    # 2. LLM流式输出（Token级别）
    elif event_type == "on_chat_model_stream":
        chunk = event["data"]["chunk"]
        if hasattr(chunk, "content") and chunk.content:
            print(chunk.content, end="", flush=True)
    
    # 3. LLM生成结束
    elif event_type == "on_chat_model_end":
        print()  # 换行
    
    # 4. 工具开始执行
    elif event_type == "on_tool_start":
        tool_name = event["name"]
        print(f"\n🔧 调用工具: {tool_name}")
    
    # 5. 工具执行结束
    elif event_type == "on_tool_end":
        tool_name = event["name"]
        print(f"✅ 工具完成: {tool_name}")

print("\n" + "="*60)
print("astream_events演示完成")
print("特点: 可以精确控制每个阶段的输出！")
print("="*60)

In [ ]:
all_events

## 7. 异步流式输出

使用`astream()`实现异步流式，适合并发场景。

In [ ]:
async def async_stream_demo():
    """异步流式演示"""
    print("="*60)
    print("异步流式输出演示")
    print("="*60)
    
    config = {"configurable": {"thread_id": "test-async"}}
    input_msg = {"messages": [HumanMessage(content="查询北京的天气")]}
    
    print("\n🤖 AI: ", end="", flush=True)
    
    # 使用astream进行异步流式
    async for msg, metadata in agent.astream(input_msg, config, stream_mode="messages"):
        if isinstance(msg, AIMessage) and msg.content:
            print(msg.content, end="", flush=True)
            # 可以在这里添加其他异步操作
            await asyncio.sleep(0)  # yield control
    
    print("\n\n✅ 异步流式完成")


# 运行异步函数
await async_stream_demo()

## 8. 流式进度追踪

实时显示Agent执行进度，包括节点转换和工具调用。

In [ ]:
def stream_with_progress():
    """带进度追踪的流式执行"""
    print("="*60)
    print("流式进度追踪演示")
    print("="*60)
    
    config = {"configurable": {"thread_id": "progress-demo"}}
    input_msg = {
        "messages": [HumanMessage(
            content="搜索'机器学习'，然后计算100*50"
        )]
    }
    
    print("\n📊 执行追踪：\n")
    
    node_count = {}
    tool_calls = []
    
    for chunk in agent.stream(input_msg, config, stream_mode="updates"):
        for node_name, node_update in chunk.items():
            # 统计节点执行次数
            node_count[node_name] = node_count.get(node_name, 0) + 1
            
            print(f"✓ 节点 [{node_name}] 执行中...")
            
            # 检查工具调用
            if "messages" in node_update:
                for msg in node_update["messages"]:
                    if hasattr(msg, 'tool_calls') and msg.tool_calls:
                        for tool_call in msg.tool_calls:
                            tool_name = tool_call['name']
                            tool_calls.append(tool_name)
                            print(f"  🔧 调用工具: {tool_name}")
    
    print("\n" + "="*60)
    print("执行统计")
    print("="*60)
    print(f"\n节点执行次数:")
    for node, count in node_count.items():
        print(f"  {node}: {count}次")
    
    print(f"\n工具调用顺序:")
    for i, tool in enumerate(tool_calls, 1):
        print(f"  {i}. {tool}")


stream_with_progress()

## 9. Steering 基础实现（15分钟）

### 第一个 Steering Demo

这是我们实现的第一个 Steering Demo，使用最简单的方式实现了流式输出和中断功能。  
因为 `steering_demo/` 已经提供了**完整实现**，包括：
- 后端(backend.py)：FastAPI + SSE + Abort 机制
- 前端(frontend.html)：原生 JS + 流式显示 + 中断按钮
- 完整文档和使用说明

**所以这里只展示核心原理，完整代码请直接运行 steering_demo 查看和学习！**

**🎯 核心公式**：

```
Steering = 流式输出 + 中断机制 + 状态恢复
```

**📌 三大核心组件**：

| 组件 | 作用 | 关键实现 |
|------|------|---------|
| **Abort Flag** | 中断信号 | 全局字典 `{session_id: bool}` |
| **Checkpointer** | 状态持久化 | `MemorySaver()` 或 `RedisSaver()` |
| **Thread ID** | 会话隔离/状态恢复 | 每个用户独立的 `thread_id` |

**💡 核心机制**：

```python
# 1. 全局中断标志
abort_flags: Dict[str, bool] = {}

# 2. 在流式输出(/api/chat)循环中检查标志
async for msg, _ in agent.astream(input, config, stream_mode="messages"):
    if abort_flags.get(session_id, False):  # 检查中断标志
        yield f"event: aborted\ndata: 已中断\n\n"
        break  # 立即停止输出

    if isinstance(msg, AIMessage) and msg.content:
        yield f"event: token\ndata: {msg.content}\n\n"

# 3. 在另一个线程(/api/abort)中设置全局中断标志
async def abort_chat(request: AbortRequest):
    session_id = request.session_id
    abort_flags[session_id] = True
```

**架构图**：

```
前端                         后端
  │                           │
  ├─ POST /api/chat ────────> 创建流式输出
  │                           ├─ abort_flags[session_id] = False
  │                           ├─ async for msg in astream():
  │                           │    if abort_flags[session_id]:
  │                           │        break  # 立即停止
  │                           └─ yield tokens...
  │                           │
  ├─ POST /api/abort ───────> 设置中断标志
  │                           abort_flags[session_id] = True
  │                           │
  ├─ 接收 event: aborted ────< 流式循环检测到标志
  └─ 显示"已中断"              break 退出循环
```


### steering_demo的局限性

**✅ 能做到**：
- 中断 LLM 的流式输出（停止显示 token）
- 保存状态（使用 checkpointer）
- 恢复对话（下次请求自动从检查点继续）

**❌ 无法做到**：
- **无法中断工具执行** - 这是最大的问题！

**🎯 问题演示**：

```python
@tool
def search_database(query: str) -> str:
    """在数据库中搜索信息（模拟2秒延迟）"""
    for i in range(10):
        print(f"🔍 搜索: {query} {i}")
        time.sleep(1)
    return f"找到关于 '{query}' 的 3 条结果：结果1、结果2、结果3"

# 用户场景：
# 1. 发送 "搜索 Python"
# 2. LLM 决定调用 search_database 工具
# 3. 工具开始执行（需要 10 秒）
# 4. 用户在第 2 秒点击🛑停止
# 5. ❌ 问题：工具仍在后台运行！(查看后台输出)
#    - abort_flags 只能停止输出循环
#    - 工具无法被打断，会一直运行到完成会一直运行到完成
```

**为什么工具无法被中断？**

工具执行位于Agent graph的一个节点中，astream开始输出时实际上已经执行完tool了，只有在astream开始输出后，我们才开始检测中断信号，换言之，在tool执行时，代码还没运行到astream的循环中，无法检测中断信号，因此不能中断工具。  

另一方面，即使将**中断信号检测**放到工具执行流程中，我们也很难控制中断的时机，比如将信号检测放在一个耗时1小时的计算任务的前或后，只要计算任务开始执行，我们就没法中断。

应该如何中断？我们继续看下一章节。



### 解决方案：异步工具 + Task 取消机制

要真正实现可中断的工具执行，需要两个核心改造：

**🔑 改造 1：将同步工具转换为异步工具**

```python
# ❌ 旧版本：同步工具（不可中断）
@tool
def search_database(query: str) -> str:
    time.sleep(5)  # 阻塞整个进程
    return "结果"

# ✅ 新版本：异步工具（可中断）
@tool
async def search_database(query: str) -> str:
    await asyncio.sleep(5)  # 这个位置可通过task.cancel()中断
    return "结果"
```

**为什么异步可以被中断？**

```
同步 time.sleep(5):
  进程 → 完全阻塞 → 什么都做不了 ❌

异步 await asyncio.sleep(5):
  任务 → 暂停当前任务 → CPU 处理其他任务 ✅
       → 可以接收取消信号 → 抛出 CancelledError ✅
```

**🔑 改造 2：使用 asyncio.Task 管理执行**

```python
# ❌ 旧版本：使用布尔标志
abort_flags = {session_id: False}

async for msg, _ in agent.astream(input, config):
    if abort_flags[session_id]:  # 只是检查标志
        break  # 工具仍在后台运行 ❌

# ✅ 新版本：使用 Task 管理
running_sessions = {}  # {session_id: asyncio.Task}

# 将 agent 执行包装成 Task
task = asyncio.create_task(run_agent())
running_sessions[session_id] = task

# 中断时，直接取消 Task
task.cancel()  # 发送 CancelledError 到工具内部 ✅
```

**核心原理图**：

```
┌─────────────────────────────────────────────────┐
│         异步可中断架构                           │
│                                                  │
│  用户点击停止                                     │
│    ↓                                             │
│  调用 /api/abort → 设置标志 + task.cancel()      │
│    ↓                                             │
│  asyncio 向 Task 注入 CancelledError             │
│    ↓                                             │
│  工具内部 await asyncio.sleep() 处捕获异常       │
│    ↓                                             │
│  except asyncio.CancelledError: ← 捕获取消信号   │
│      return "操作被用户中断"  ← 优雅退出         │
│    ↓                                             │
│  Agent 立即停止，释放资源 ✅                      │
│    ↓                                             │
│  处理用户的新消息 ✅                              │
│                                                  │
└─────────────────────────────────────────────────┘
```

### 小结

steering_demo 实现了 Steering 的基础框架：
- ✅ 流式输出
- ✅ 中断信号机制（abort_flags）
- ✅ 状态持久化（checkpointer）

但存在关键缺陷：
- ❌ 无法真正中断工具执行
- ❌ 工具会在后台继续运行，浪费资源

解决方案：异步工具 + Task 取消机制  
**完整实现会放在后续的高级教程中**

## 10. 核心概念总结

### 四种Stream模式对比

| 模式 | 返回内容 | 粒度 | 适用场景 |
|------|---------|------|----------|
| values | 完整状态 | 节点级 | 调试、日志记录 |
| updates | 增量更新 | 节点级 | 进度追踪、传输优化 |
| messages | Token流 | Token级 | 聊天界面、实时打字效果 |
| astream_events | 事件流 | 事件级 | 生产环境、精确控制 |

### astream_events vs messages

```python
# messages模式：只能获取Token流
for msg, metadata in agent.stream(..., stream_mode="messages"):
    print(msg.content, end="")  # 只有内容

# astream_events：可以监听所有事件
async for event in agent.astream_events(..., version="v2"):
    if event["event"] == "on_tool_start":
        print(f"工具开始: {event['name']}")  # ✅ 可以知道工具何时开始
    elif event["event"] == "on_chat_model_stream":
        print(event["data"]["chunk"].content, end="")
```

### 流式输出的优势

1. **用户体验**: 实时反馈 vs 黑盒等待
2. **可观测性**: 清晰看到执行过程
3. **性能感知**: 降低等待的心理时间


### 同步 vs 异步

```python
# 同步流式
for chunk in agent.stream(...):
    process(chunk)

# 异步流式（推荐用于并发场景）
async for chunk in agent.astream(...):
    await async_process(chunk)

# astream_events（必须异步）
async for event in agent.astream_events(..., version="v2"):
    handle_event(event)
```

### steering机制
1. 一个简单的steering_demo, backend.py+frontend.html, 展示输出流中断。
2. 一个真正的steering机制的解决方案。

## 11. 练习题

### 练习1: 自定义事件过滤器
实现一个事件过滤器，只显示特定类型的事件（如只显示工具调用事件）。

**提示**:
```python
async def filtered_stream(agent, input_msg, config, event_types: List[str]):
    """只输出指定类型的事件"""
    async for event in agent.astream_events(input_msg, config, version="v2"):
        if event["event"] in event_types:
            yield event
```

### 练习2: 流式进度条
使用 `ipywidgets` 实现一个真实的进度条，实时显示Agent执行进度。

**提示**:
```python
import ipywidgets as widgets
from IPython.display import display

progress = widgets.IntProgress(min=0, max=100, description='执行中:')
display(progress)

# 在流式循环中更新进度
progress.value = current_step / total_steps * 100
```

### 练习3: 带超时的异步工具
实现一个带超时机制的异步工具，如果执行超过指定时间自动中断。

**提示**:
```python
@tool
async def search_with_timeout(query: str, timeout: int = 5) -> str:
    """带超时的搜索工具"""
    try:
        result = await asyncio.wait_for(
            actual_search(query), 
            timeout=timeout
        )
        return result
    except asyncio.TimeoutError:
        return f"搜索超时（{timeout}秒）"
```

### 练习4: 实现真正的可中断工具
参考第9章的解决方案，实现一个使用 `asyncio.Task` 的完整可中断系统。

**核心步骤**:
1. 将所有工具改为异步
2. 使用 `asyncio.create_task()` 包装执行
3. 在中断时调用 `task.cancel()`
4. 在工具内部捕获 `CancelledError`

### 思考题
   
1. **如何平衡流式输出的实时性和性能开销？**
   - 提示：批量发送 vs 逐个发送的权衡

2. **在多用户场景下，如何确保每个用户的流式输出互不干扰？**
   - 提示：思考 `thread_id` 和 `session_id` 的作用

3. **如果工具执行失败，中断机制应该如何处理？**
   - 保存中间状态？重试？直接返回错误？


## 12. 下一步

通过前面的学习，你已经掌握了构建智能Agent的核心技能：
- ✅ 第1章：基础ReAct Agent - 工具调用
- ✅ 第2章：人类反馈 - 中断和审批
- ✅ 第3章：子Agent - 模块化设计
- ✅ 第4章：任务管理 - Todo系统
- ✅ 第5章：上下文压缩 - 长对话处理
- ✅ 第6章：流式输出和中断 - 实时响应

接下来建议:
1. 探索本项目目录下的claude_code_demo，该demo整合了上面所有的功能特征。请尝试运行、调试、改造该项目。
2. 思考距离真正的claude code还有多大差距，你可以参考项目目录下的补充材料[`supp1`](supp1_from_course_to_production.md)。

Happy Learning! 🚀